# Libraries

## Import

In [1]:
# Libraries

import sys
sys.path.append('../data/ma-bench/')
sys.path.append('../data/tau-bench/')

import os
import json
import importlib
import argparse
import base64
import uuid

import boto3
from botocore.config import Config

# Strands imports
from strands import Agent, tool
from strands.models import BedrockModel
from strands.multiagent import GraphBuilder
from strands.telemetry.config import StrandsTelemetry

# Parameters

In [2]:
# setup boto3 config to allow for retrying
region_name = "us-east-1"
my_config = Config(
    region_name = region_name,
    signature_version = 'v4',
    retries = {
        'max_attempts': 50,
        'mode': 'standard'
    }
)

# select domain
domain = "airline"
# # Parse command line arguments
# parser = argparse.ArgumentParser(description='Run agent with specified domain')
# parser.add_argument('--domain', type=str, default=domain, 
#                     help='Domain to use (e.g., "airline", "retail")')
# args = parser.parse_args()

# # Update domain if provided via command line
# domain = args.domain


######################### LANGFUSE SETUP ########################
# Langfuse credentials
os.environ["LANGFUSE_PUBLIC_KEY"] = "[ADD PUBLIC KEY HERE]"
os.environ["LANGFUSE_SECRET_KEY"] = "[ADD SECRET KEY HERE]"
os.environ["LANGFUSE_HOST"] = "https://us.cloud.langfuse.com"

# Build Basic Auth header
LANGFUSE_AUTH = base64.b64encode(
    f"{os.environ.get('LANGFUSE_PUBLIC_KEY')}:{os.environ.get('LANGFUSE_SECRET_KEY')}".encode()
).decode()

# Configure OpenTelemetry endpoint & headers
os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = os.environ.get("LANGFUSE_HOST") + "/api/public/otel/"
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {LANGFUSE_AUTH}"

# Initialize OpenTelemetry BEFORE creating Strands agent

strands_telemetry = StrandsTelemetry()
strands_telemetry.setup_otlp_exporter()
# strands_telemetry.setup_console_exporter()  # Print traces to console
######################### LANGFUSE SETUP ########################

# Utils

In [3]:
def import_domain_tools(domain):
    """
    Dynamically import tools based on the domain
    """
    tools_module = importlib.import_module(f'mabench.environments.{domain}.tools_strands')
    tools_dict = {}
    
    # Get all attributes from the tools module
    for attr_name in dir(tools_module):
        if attr_name.startswith('__'):
            continue
        
        try:
            # Try to import each tool
            tool_module = importlib.import_module(f'mabench.environments.{domain}.tools_strands.{attr_name}')
            # Get the tool function from the module
            if hasattr(tool_module, attr_name):
                tools_dict[attr_name] = getattr(tool_module, attr_name)
        except (ImportError, AttributeError):
            pass
    
    return tools_dict

In [4]:
# Import domain-specific modules
try:
    # Import wiki
    wiki_module = importlib.import_module(f'tau_bench.envs.{domain}.wiki')
    WIKI = getattr(wiki_module, 'WIKI')
    
    # Import data and tasks
    importlib.import_module(f'tau_bench.envs.{domain}.data')
    importlib.import_module(f'tau_bench.envs.{domain}.tasks')
    
    # Import tools
    domain_tools = import_domain_tools(domain)
    
    print(f"Successfully loaded modules for domain: {domain}")
except ImportError as e:
    print(f"Error: Could not import modules for domain '{domain}'. Error: {e}")
    print("Available domains may include: airline, retail")
    sys.exit(1)

Successfully loaded modules for domain: airline


# Agent

In [5]:
tools = list(domain_tools.values())

def agent_prompt():
    
    system_prompt_template = """
You are a helpful assistant for a travel website. Help the user answer any questions.

<policy>
{policy}
</policy>

<instructions>
- Remeber to check if the the airport city is in the state mentioned by the user. For example, Houston is in Texas.
- Infer about the the U.S. state in which the airport city resides. For example, Houston is in Texas.
- You should not use made-up or placeholder arguments.
- Do not ask for any confirmation from the user. Just go ahead and execute your actions.
- Do not ask the user if they want you to proceed or not.
<instructions>
"""

    prompt = system_prompt_template.format(policy = WIKI)

    return prompt


def agent_model():

    model_id = "anthropic.claude-3-sonnet-20240229-v1:0" # "anthropic.claude-3-sonnet-20240229-v1:0" "anthropic.claude-3-5-sonnet-20240620-v1:0", "us.anthropic.claude-3-5-sonnet-20241022-v2:0" 

    return BedrockModel(
        model_id = model_id,
        region_name = region_name,
        max_tokens= 1024,
        temperature = 0.0,
        top_p = 1,
        boto_client_config=my_config,
    )


def agent_tracing(user_id, session_id, domain):

    trace_attributes = {
        "user.id": user_id, 
        "session.id": session_id,
        "langfuse.tags": [
            user_id,
            session_id,
            f"awsStrands-singleAgent_singleTurn-{domain}",
        ]
    }

    return trace_attributes


def react_agent(tools, user_id, session_id, domain):

    prompt = agent_prompt()
    model = agent_model()
    trace_attributes = agent_tracing(user_id, session_id, domain)

    return Agent(
        name = f"awsStrands-singleAgent_singleTurn-{domain}-{user_id}-{session_id}",
        model = model, 
        tools = tools, 
        system_prompt = prompt,
        trace_attributes = trace_attributes
    )

# Run

In [6]:
output_path = os.path.join("..", "data", "tau-bench", "tau_bench", "envs", f"{domain}", "tasks_singleturn.json")
with open(output_path, "r") as file:
    tasks = json.load(file)


In [7]:
# for index,task in enumerate(tasks):

index = 19
task = tasks[index]

index_str = str(index)
num_hashes = (50 - len(index_str) - 9) // 2
print(f"\n{'#' * num_hashes} Index:{index} {'#' * num_hashes}\n")

question = task['question']
print(f"Processing question: {question}")

user_id = task['user_id']
session_id= uuid.uuid4()
print(f"User ID: {user_id}\tSession ID: {session_id}\tDomain:{domain}")

agent = react_agent(tools, user_id, session_id, domain)

messages = agent(question)
print(messages)

    # break


################### Index:19 ###################

Processing question: 
My user id is raj_brown_5782. I want to change my upcoming roundtrip flights which are currently DTW to LGA and back (reservation ID is VA5SGQ). I want to change them to nonstop flights from DTW to JFK and back on the same dates as the current reservation. Since I took insurance for this trip, I want change fees waived. I also want to add 1 checked bag. I prefer to choose morning flights that arrive before 7am at the destination and then also want to choose the cheapest Economy (not Basic Economy) options within those constraints.

User ID: raj_brown_5782	Session ID: 85681a07-dc3a-4dba-b904-760ec3b55dea	Domain:airline
Okay, let me check the details of your existing reservation first.
Tool #1: get_reservation_details


Based on the details, your current reservation is a round-trip from DTW to LGA with a stop in PHX on both ways. The flights are on May 17 and May 19-20. You have economy cabin, no checked bags, and t

In [8]:
langfuse_trace_id = hex(agent.trace_span.context.trace_id)[2:]
langfuse_trace_id

'8e6cf12d11608ecb1da9d02bfd8d1d34'